# Agentic RAG with LangGraph

In [9]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode
from langchain.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Literal
import os

print("All imports successful")

All imports successful


In [2]:
load_dotenv()
api_key = os.getenv("paid_api")

if not api_key:
    raise ValueError("API_Key not found. Please set it in your .env file")
print("API key loaded")

API key loaded


In [3]:
## Initialize LLM
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.5,
    api_key = api_key
)
print(f"LLM initialized: {llm.model_name}")

LLM initialized: gpt-4o-mini


## Load and Process Documents

In [5]:
file_path = r"c:\Users\owner\Downloads\Article_Research\Artificial Intelligence in the Energy Industry.pdf"

# Check if file exists
if not os.path.exists(file_path):
    print(f"File not found {file_path}")
    print("Please update the file_path variable with your PDF file.")

    #Creating sample documents for demo
    from langchain_core.documents import Document
    pages = [
        Document(page_content="Biochemistry is the study of chemical processes in living organisms.",
                 metadata={"page":1}),
        Document(page_content= "Proteins are made of amino acids and perform many functions in cells.",
                 metadata = {"page":2}),
        Document(page_content="DNA stores genetic information using four nucleotide bases.",
                 metadata={"page":3})
    ]
    print("Using sample documents for demo")
else:
    #Load the pdf
    loader = PyPDFLoader(file_path)
    pages= []

    #Load pages (async loading)
    async for page in loader.alazy_load(): # async prevents memory overload by processing pages from a PDF one by one instead of all at once
        pages.append(page)
    print(f"Loaded {len(pages)} pages from PDF.")

Loaded 11 pages from PDF.


### Split into Chunks

In [6]:
# Create text_splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap=100
)

#Split documents
doc_splits = text_splitter.split_documents(pages)

print(f"Created {len(doc_splits)} chunks")
print(f"\n Sample chunk:")
print(f"{doc_splits[0].page_content[:200]}...")

Created 22 chunks

 Sample chunk:
What is Artiﬁcial Intelligence in
the Energy Industry ?
Deﬁnition
In recent years, Artiﬁcial Intelligence (AI) has gained relevance in a wide variety of
sectors. However, deﬁning the term poses some d...


## Create Vector Store (Chromadb)

In [7]:
#Initilize embeddings
embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    api_key=api_key
)
print("Embeddings model initialized")

Embeddings model initialized


In [10]:
# Create Chroma vector stor
chroma_path = "./chroma_db_agentic_rag"

#Create vector store from documents
vectorstore = Chroma(
    collection_name="agentic_rag_docs",
    persist_directory=chroma_path,
    embedding_function=embeddings
)

#Add documents
vectorstore.add_documents(documents=doc_splits)
print(f"Vector store created with {len(doc_splits)} chunks")
print(f"Persisted to: {chroma_path}")

Vector store created with 22 chunks
Persisted to: ./chroma_db_agentic_rag


### Test Retrieval

In [11]:
# Test the vector store
test_query = "What is artificial intelligence?"
test_results = vectorstore.similarity_search(test_query, k=2)

print(f"Query: {test_query}")
print(f"\nTop result:")
print(f"{test_results[0].page_content[:200]}...")
print(f"\nRetrieval working!")

Query: What is artificial intelligence?

Top result:
Even with this deﬁnition, the term remains diﬃcult to grasp. In practice, therefore,
it is common to speak of strong or weak AI. A strong AI is one in which the
application has all aspects associated ...

Retrieval working!


## Create Retrieval Tool

In [13]:
@tool
def retrieve_documents(query: str) -> str:
    """
    Search for relevant documents in the knowledge base.
    
    Use this tool when you need information from the document collection
    to answer the user's question. Do NOT use this for:
    - General knowledge questions
    - Greetings or small talk
    - Simple calculations
    
    Args:
        query: The search query describing what information is needed
        
    Returns:
        Relevant document excerpts that can help answer the question
    """
    # Use MMR (Maximum Marginal Relevance) for diverse results
    retriever = vectorstore.as_retriever(
        search_type = "mmr",
        search_kwargs ={"k":5, "fetch_k":10}
    )

    # Retrieve documents
    results = retriever.invoke(query)

    if not results:
        return "No relevant documents found"
    
    formatted = "\n\n---\n\n".join(
        f"Document {i+1}:\n{doc.page_content}"
        for i, doc in enumerate(results)
    )
    return formatted
print("Retrieval tool created")


Retrieval tool created


### Test the Tool

In [15]:
# Test tool directly
test_result = retrieve_documents.invoke({"query": "What is artificial intelligence?"})
print(f"Tool result (first 300 chars): \n{test_result[:300]}...")

Tool result (first 300 chars): 
Document 1:
Even with this deﬁnition, the term remains diﬃcult to grasp. In practice, therefore,
it is common to speak of strong or weak AI. A strong AI is one in which the
application has all aspects associated with human intelligence, such as the ability
to draw logical conclusions, the existence ...


## Build the Agentic RAG System

### System Prompt

In [16]:
system_prompt = SystemMessage(content="""You are a helpful assistant with access to a document retrieval tool.

RETRIEVAL DECISION RULES:

DO NOT retrieve for:
- Greetings: "Hello", "Hi", "How are you"
- Questions about your capabilities: "What can you help with?", "What do you do?"
- Simple math or general knowledge: "What is 2+2?"
- Casual conversation: "Thank you", "Goodbye"

DO retrieve for:
- Questions asking for specific information that would be in documents
- Requests for facts, definitions, or explanations about specialized topics
- Any question where citing sources would improve the answer

Rule of thumb: If the user is asking for information (not just chatting), retrieve first.

When you retrieve documents, cite them in your answer. If documents don't contain the answer, say so.
""")

print("System prompt configured")

System prompt configured


### Agent Nodes

In [ ]:
# Bind tool to LLM
tools = [retrieve_documents]
llm_with_tools = llm.bind_tools(tools)

def assistant(state: MessagesState) -> dict:
    """
    Assistant node - decides whether to retrieve or answer directly.
    """
    messages = [system_prompt] + state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]: